In [ ]:
'''
Setup
1. Import Library
2. Import File
3. Cleaning data
4. Feature Engineering
5. Buat Model
6. Tes Model
'''

In [ ]:
#Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from mlxtend.preprocessing import minmax_scaling

print("Setup done")

Setup done


In [66]:
#Import File
data= pd.read_csv("C:/Users/visco/Downloads/Housing Prize/train.csv")
print(data.shape)
data.iloc[:,20:30]

(1460, 81)


,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation
0,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc
1,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock
2,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc
3,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil
4,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc
...,...,...,...,...,...,...,...,...,...,...
1455,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc
1456,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock
1457,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone
1458,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock


In [3]:
#Liat data hilang
data_hilang = data.isnull().sum()
data_hilang[70:81]

ScreenPorch         0
PoolArea            0
PoolQC           1453
Fence            1179
MiscFeature      1406
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
dtype: int64

In [55]:
#Hapus data Hilang
data_bersih = data.dropna(axis= 1) #Hapus kolom yang punya missing values
print(f'Columns Before Data Cleaning: {data.shape[1]}')
print(f'Columns After Data Cleaning: {data_bersih.shape[1]}')
print(data_bersih.shape)


Columns Before Data Cleaning: 81
Columns After Data Cleaning: 62
(1460, 62)


In [56]:
#Kolom yang dihapus
colbf= data.columns
colaf=data_bersih.columns

dropped_cols = set(colbf)-set(colaf)
print(dropped_cols)

{'GarageType', 'Electrical', 'GarageFinish', 'LotFrontage', 'BsmtQual', 'BsmtExposure', 'PoolQC', 'MiscFeature', 'BsmtCond', 'GarageYrBlt', 'FireplaceQu', 'Alley', 'GarageCond', 'GarageQual', 'MasVnrArea', 'BsmtFinType2', 'Fence', 'BsmtFinType1', 'MasVnrType'}


In [27]:
#Cek Kolom
print(data['Functional'])

0        Typ
1        Typ
2        Typ
3        Typ
4        Typ
        ... 
1455     Typ
1456    Min1
1457     Typ
1458     Typ
1459     Typ
Name: Functional, Length: 1460, dtype: object


In [ ]:
#One-Hot Decoding
data_bersih['MSSubClass'] = data_bersih['MSSubClass'].astype('category')
data_bersih = pd.get_dummies(data_bersih ,columns= ['MSSubClass'], prefix='MSSubClass')
data_bersih['BldgType'] = data_bersih['BldgType'].astype('category')
data_bersih = pd.get_dummies(data_bersih ,columns= ['BldgType'], prefix='BldgType')
data_bersih['HouseStyle'] = data_bersih['HouseStyle'].astype('category')
data_bersih = pd.get_dummies(data_bersih ,columns= ['HouseStyle'], prefix='HouseStyle')

(1460, 87)


C:\Users\visco\AppData\Local\Temp\ipykernel_22068\2104143978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bersih['MSSubClass'] = data_bersih['MSSubClass'].astype('category')


In [59]:
#Mapping
map_exter = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
data_bersih['ExterQual'] = data_bersih['ExterQual'].map(map_exter)
data_bersih['ExterCond'] = data_bersih['ExterCond'].map(map_exter)
data_bersih['KitchenQual'] = data_bersih['KitchenQual'].map(map_exter) #Sama


In [67]:
#cek kolom after one-hot
print(f'Dimensi Data setelah One-Hot: {data_bersih.shape}')
[col for col in data_bersih.columns if 'HouseStyle' in col]

Dimensi Data setelah One-Hot: (1460, 87)


['HouseStyle_1.5Fin',
 'HouseStyle_1.5Unf',
 'HouseStyle_1Story',
 'HouseStyle_2.5Fin',
 'HouseStyle_2.5Unf',
 'HouseStyle_2Story',
 'HouseStyle_SFoyer',
 'HouseStyle_SLvl']

In [63]:
#Pemilihan Kolom untuk train_data
y = data_bersih.SalePrice
col_train = ['MSSubClass_20',
             'MSSubClass_30',
             'MSSubClass_40',
             'MSSubClass_45',
             'MSSubClass_50',
             'MSSubClass_60',
             'MSSubClass_70',
             'MSSubClass_75',
             'MSSubClass_80',
             'MSSubClass_85',
             'MSSubClass_90',
             'MSSubClass_120',
             'MSSubClass_160',
             'MSSubClass_180',
             'MSSubClass_190', 
             'BldgType_1Fam',
             'BldgType_2fmCon',
             'BldgType_Duplex',
             'BldgType_Twnhs',
             'BldgType_TwnhsE', 
             'HouseStyle_1.5Fin',
             'HouseStyle_1.5Unf',
             'HouseStyle_1Story',
             'HouseStyle_2.5Fin',
             'HouseStyle_2.5Unf',
             'HouseStyle_2Story',
             'HouseStyle_SFoyer',
             'HouseStyle_SLvl', 
             'OverallQual', 
             'OverallCond', 
             'YearBuilt',
             'YearRemodAdd',
             'ExterQual', 
             'ExterCond', 
             'HeatingQC',
             'GrLivArea',
             'FullBath',
             'HalfBath',
             'BedroomAbvGr',
             'KitchenAbvGr',
             'KitchenQual', 
             'GarageCars',
             'GarageArea',
             'WoodDeckSF',
             'OpenPorchSF',
             'ScreenPorch',
             'PoolArea',
             'YrSold']
X = data_bersih[col_train]

In [ ]:
#train
train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.25, random_state=1)
model = RandomForestClassifier(max_depth = 10, random_state = 1)

model.fit(train_X,train_y)
vald_predict = model.predict(val_X)
